In [62]:
from langchain_ollama import OllamaLLM
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import SystemMessage, HumanMessage

load_dotenv()


True

In [38]:
model = OllamaLLM(model="llama3.2")

#### Default Tools

In [39]:
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun, YouTubeSearchTool


In [4]:
api_wrapper = WikipediaAPIWrapper()
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)
print(f"Tool name: {wiki_tool.name}, Tool description: {wiki_tool.description}, Tool Arguments: {wiki_tool.args}")
wiki_tool.run("LSTM model")

Tool name: wikipedia, Tool description: A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query., Tool Arguments: {'query': {'description': 'query to look up on wikipedia', 'title': 'Query', 'type': 'string'}}


'Page: Long short-term memory\nSummary: Long short-term memory (LSTM) is a type of recurrent neural network (RNN) aimed at mitigating the vanishing gradient problem commonly encountered by traditional RNNs. Its relative insensitivity to gap length is its advantage over other RNNs, hidden Markov models, and other sequence learning methods. It aims to provide a short-term memory for RNN that can last thousands of timesteps (thus "long short-term memory"). The name is made in analogy with long-term memory and short-term memory and their relationship, studied by cognitive psychologists since the early 20th century.\nAn LSTM unit is typically composed of a cell and three gates: an input gate, an output gate, and a forget gate. The cell remembers values over arbitrary time intervals, and the gates regulate the flow of information into and out of the cell. Forget gates decide what information to discard from the previous state, by mapping the previous state and the current input to a value be

In [7]:
yt_tool = YouTubeSearchTool()
yt_tool.run("Krish Naik")

"['https://www.youtube.com/watch?v=zDKTnYDbMkE&pp=ygUKS3Jpc2ggTmFpaw%3D%3D', 'https://www.youtube.com/watch?v=xOS0BhhdUbo&pp=ygUKS3Jpc2ggTmFpaw%3D%3D']"

#### Customizing Default Tools

In [10]:
from pydantic import BaseModel, Field

class CustomWikiInputs(BaseModel):
    """ Inputs to wikipedia tool """
    query: str = Field(description="query to look up in wikipedia, should not be more than 3 words")

custom_wiki_tool = WikipediaQueryRun(
    name="Wiki Tool",
    description="Searches for contnet of any query topic in wikipedia if available",
    args_schema=CustomWikiInputs,
    api_wrapper=api_wrapper,
    return_direct=True
)
custom_wiki_tool.run("What is ensemble technique in mL?")


"Page: Random matrix\nSummary: In probability theory and mathematical physics, a random matrix is a matrix-valued random variable—that is, a matrix in which some or all of its entries are sampled randomly from a probability distribution. Random matrix theory (RMT) is the study of properties of random matrices, often as they become large. RMT provides techniques like mean-field theory, diagrammatic methods, the cavity method, or the replica method to compute quantities like traces, spectral densities, or scalar products between eigenvectors. Many physical phenomena, such as the spectrum of nuclei of heavy atoms, the thermal conductivity of a lattice, or the emergence of quantum chaos, can be modeled mathematically as problems concerning large, random matrices.\n\nPage: Mixture of experts\nSummary: Mixture of experts (MoE) is a machine learning technique where multiple expert networks (learners) are used to divide a problem space into homogeneous regions. MoE represents a form of ensembl

### Agents

In [65]:
from langchain.agents import AgentType, AgentExecutor, initialize_agent, load_tools, create_tool_calling_agent, create_react_agent, create_self_ask_with_search_agent
from langchain.tools.tavily_search import TavilySearchResults
from langchain import hub
from langchain.tools import tool, Tool
from langchain.memory import ConversationBufferMemory
from langchain.utilities import GoogleSerperAPIWrapper



#### Tool Calling Agent

In [15]:

tool = load_tools(["wikipedia"], llm=model)
agent = initialize_agent(tools=tool, llm = model, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose = True)
agent.run("Who won USA presidential election 2016?")



> Entering new AgentExecutor chain...
Thought: To find out who won the USA presidential election in 2016, I should look up information on that event.

Action: wikipedia
Action Input: usa presidential election 2016
Observation: Page: 2016 United States presidential election
Summary: Presidential elections were held in the United States on November 8, 2016. The Republican ticket of businessman Donald Trump and Indiana governor Mike Pence defeated the Democratic ticket of former secretary of state and former first lady Hillary Clinton and Virginia junior senator Tim Kaine, in what was considered one of the biggest political upsets in American history. It was the fifth and most recent presidential election in which the winning candidate lost the popular vote. It was also the sixth and most recent presidential election in U.S. history in which both major party candidates were registered in the same home state.
Incumbent Democratic president Barack Obama was ineligible to pursue a third te

'Donald Trump and Mike Pence won the USA presidential election in 2016.'

In [12]:
search = TavilySearchResults()
search.run("Who won the border gavaskar tropy 24-25?")
# tools = [search]
# prompt_template = ChatPromptTemplate.from_messages(
#     [
#         SystemMessage("You use available tool to search from the internet about any topic"),
#         MessagesPlaceholder("chat_history"),
#         ("human", "{input}"),
#         MessagesPlaceholder(variable_name="agent_scratchpad")
#     ]
# )
# agent = create_tool_calling_agent(tools=tools, llm=model, prompt=prompt_template)
# agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
# agent_executor.invoke({"input" : "Who won the border gavaskar tropy 24-25?"})

[{'url': 'https://www.ipl.com/cricket/news/australia-defeats-india-3-1-to-win-border-gavaskar-trophy-after-a-decade/',
  'content': '22 hours ago ... Australia won the Border-Gavaskar Trophy back by beating India by 6 wickets in the last of five Test matches on Sunday at the Sydney Cricket'},
 {'url': 'https://timesofindia.indiatimes.com/sports/cricket/india-in-australia/boom-magic-to-record-crowds-five-big-talking-points-from-2024/25-border-gavaskar-trophy/articleshow/116965280.cms',
  'content': "17 hours ago ... Australia clinched the Border-Gavaskar Trophy with a 3-1 series victory over India, highlighted by Scott Boland's exceptional bowling. Despite"},
 {'url': 'https://www.nine.com.au/sport/cricket/australia-vs-india-fifth-test-2025-day-three-recap-highlights-beau-webster-winning-runs-border-gavaskar-trophy-history-20250105-p5l26n.html',
  'content': '4 hours ago ... The drought is over - Australia has won the Border-Gavaskar Trophy for the first time in 10 years. The hosts tota

#### ReAct (Reasoning & Action) Agent

In [56]:
api_wrapper = WikipediaAPIWrapper()
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

@tool
def get_employee_id_by_name(name: str):
    """
        Get employee id given the employee name from the employee database
    """
    fake_employee_data = {
        "Alex": "E001",
        "Ben": "E002",
        "Clarke": "E003",
        "Dean": "E004",
        "Emily": "E005",
        "Frank": "E006",
        "Gus": "E007",
        "Henry": "E008"
    }
    return fake_employee_data.get(name.strip(), "Employee name does't exist")

@tool
def get_employee_sal_by_id(emp_id: str):
    """
        Get employee salary given the employee id from the employee database
    """
    fake_employee_data = {
         "E001": 23000,
         "E002": 34000,
         "E003": 27000,
         'E004': 28000,
         "E005": 39000,
         "E006": 45000,
         "E007": 21000,
         "E008": 36000
     }
    return fake_employee_data.get(emp_id.strip(), "Employee id not found")

tools = [get_employee_id_by_name, get_employee_sal_by_id, wiki_tool]
prompt = hub.pull("hwchase17/react")
print(f"=====\n{prompt}\n=====\n")
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
memory.chat_memory.add_messages(("system", "You are an AI assistant, who uses available tools to answer questions. If you don't know the answer, reply with 'I don't know'."))

agent = create_react_agent(tools=tools, prompt=prompt, llm=model)
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)
agent_executor.invoke({"input": input("You: ")})

/home/bishwayansaha99/langchain/venv/lib/python3.12/site-packages/langsmith/client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


=====
input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'} template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'
=====



> Entering new AgentExecutor chain...
Question: What is salary of Ben?
Thought: I need to get the employee ID by name, 

/home/bishwayansaha99/langchain/venv/lib/python3.12/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /home/bishwayansaha99/langchain/venv/lib/python3.12/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Page: Ben
Summary: Ben is frequently used as a shortened version of the given names Benjamin, Benedict, Bennett, Benson or Ebenezer, and is also a given name in its own right.
Ben meaning "son of" is also found in Arabic as Ben (dialectal Arabic) or  bin (بن), Ibn/ebn (ابن).
Ben (賁/便嗯
) is a Chinese surname.



Page: Ben Affleck
Summary: Benjamin Géza Affleck (born August 15, 1972) is an American actor and filmmaker. His accolades include two Academy Awards, two BAFTA Awards, and three Golden Globes. Affleck began his career as a child when he starred in the PBS educational series The Voyage of the Mimi (1984–1988). He later appeared in the independent comedy Dazed and Confused (1993) and several Kevin Smith comedies, including Chasing Amy (1997).
Affleck gained wider recognition when he and Matt Damon won the Academy Award for Best Original Screenplay for writing Good Will Hunting (1997), which they also starred in. He established himself as a leading man in studio films, including th

/home/bishwayansaha99/langchain/venv/lib/python3.12/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /home/bishwayansaha99/langchain/venv/lib/python3.12/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Page: Ben
Summary: Ben is frequently used as a shortened version of the given names Benjamin, Benedict, Bennett, Benson or Ebenezer, and is also a given name in its own right.
Ben meaning "son of" is also found in Arabic as Ben (dialectal Arabic) or  bin (بن), Ibn/ebn (ابن).
Ben (賁/便嗯
) is a Chinese surname.



Page: Ben Affleck
Summary: Benjamin Géza Affleck (born August 15, 1972) is an American actor and filmmaker. His accolades include two Academy Awards, two BAFTA Awards, and three Golden Globes. Affleck began his career as a child when he starred in the PBS educational series The Voyage of the Mimi (1984–1988). He later appeared in the independent comedy Dazed and Confused (1993) and several Kevin Smith comedies, including Chasing Amy (1997).
Affleck gained wider recognition when he and Matt Damon won the Academy Award for Best Original Screenplay for writing Good Will Hunting (1997), which they also starred in. He established himself as a leading man in studio films, including th

/home/bishwayansaha99/langchain/venv/lib/python3.12/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /home/bishwayansaha99/langchain/venv/lib/python3.12/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Page: Ben
Summary: Ben is frequently used as a shortened version of the given names Benjamin, Benedict, Bennett, Benson or Ebenezer, and is also a given name in its own right.
Ben meaning "son of" is also found in Arabic as Ben (dialectal Arabic) or  bin (بن), Ibn/ebn (ابن).
Ben (賁/便嗯
) is a Chinese surname.



Page: Ben Affleck
Summary: Benjamin Géza Affleck (born August 15, 1972) is an American actor and filmmaker. His accolades include two Academy Awards, two BAFTA Awards, and three Golden Globes. Affleck began his career as a child when he starred in the PBS educational series The Voyage of the Mimi (1984–1988). He later appeared in the independent comedy Dazed and Confused (1993) and several Kevin Smith comedies, including Chasing Amy (1997).
Affleck gained wider recognition when he and Matt Damon won the Academy Award for Best Original Screenplay for writing Good Will Hunting (1997), which they also starred in. He established himself as a leading man in studio films, including th

{'input': 'What is salary of Ben?',
 'output': 'Agent stopped due to iteration limit or time limit.'}

#### Self Ask Agent

In [73]:
google_search = GoogleSerperAPIWrapper()
tools = [
    Tool(
        name="Intermediate Answer",
        func=google_search.run,
        description="Useful when you want to search about any topic",
        verbose=True
    )
]
prompt = hub.pull("hwchase17/self-ask-with-search")
print(f"=====\n{prompt}\n=====\n")
agent = create_self_ask_with_search_agent(llm=model, tools=tools, prompt=prompt)
agent_executor = AgentExecutor.from_agent_and_tools(tools=tools, verbose=True, agent=agent, handle_parsing_errors=True)
agent_executor.invoke({"input": "What is better player, Ronaldo or Messi?"})

/home/bishwayansaha99/langchain/venv/lib/python3.12/site-packages/langsmith/client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


=====
input_variables=['agent_scratchpad', 'input'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'self-ask-with-search', 'lc_hub_commit_hash': '9ef33d1fc82608bdfaa88b3757306ec4f22511b736cb267b1737616641dac0cc'} template='Question: Who lived longer, Muhammad Ali or Alan Turing?\nAre follow up questions needed here: Yes.\nFollow up: How old was Muhammad Ali when he died?\nIntermediate answer: Muhammad Ali was 74 years old when he died.\nFollow up: How old was Alan Turing when he died?\nIntermediate answer: Alan Turing was 41 years old when he died.\nSo the final answer is: Muhammad Ali\n\nQuestion: When was the founder of craigslist born?\nAre follow up questions needed here: Yes.\nFollow up: Who was the founder of craigslist?\nIntermediate answer: Craigslist was founded by Craig Newmark.\nFollow up: When was Craig Newmark born?\nIntermediate answer: Craig Newmark was born on December 6, 1952.\nSo the final answer is: December 6, 1952\n\nQuest

{'input': 'What is better player, Ronaldo or Messi?',
 'output': 'Agent stopped due to iteration limit or time limit.'}

#### Custom ReAct Agent

In [ ]:
class ChatBot():
    def __init__(self):
        pass

    def __call__(self, *args, **kwds):
        pass
    
    def execute(self):
        pass

prompt = """
"""